In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import randint

import dataframe_image as dfi

import json
import joblib
import pickle

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve, roc_auc_score

In [2]:


def get_metrics(y_test, y_pred):
    predicted_for_discharge = list(np.where(y_pred == False)[0])
    wrongful_discharge = y_test.reset_index(drop=True)[predicted_for_discharge].sum()/len(predicted_for_discharge)
    print(f"WRONGFUL DISCHARGE RATE: {wrongful_discharge}")

    print(f"F1_SCORE: {f1_score(y_test, y_pred)}")
    print(f"RECALL: {recall_score(y_test, y_pred)}")
    print(f"PRECISION: {precision_score(y_test, y_pred)}")

    

    try:
        print(f"ROC AUC: {roc_auc_score(y_test, y_pred)}")
    except: "ROC AUC curve could not be calculated"


In [3]:
#data = pd.read_csv("data/cleaned_data_add_features.csv")
data = pd.read_csv("data/cleaned_data.csv")

#data = pd.read_csv("data/train_data.csv")
#data.readmitted = data.readmitted.replace(["Yes", "No"], [True, False])
#data[categorical_features] =  data[categorical_features].astype(str)
#data = data[~data["discharge_disposition_code"].isin(["11.0", "13.0", "14.0", "19.0", "20.0", "21.0"])]

## change target value to boolean
data.readmitted = data.readmitted.replace(["Yes", "No"], [True, False])

data.head()

,admission_id,patient_id,race,gender,age,weight,admission_type_code,discharge_disposition_code,admission_source_code,time_in_hospital,...,blood_type,hemoglobin_level,blood_transfusion,max_glu_serum,A1Cresult,diuretics,insulin,change,diabetesMed,readmitted
0,0,199042938,Caucasian,Male,[50-60),NaN,3.0,1.0,1.0,1,...,A+,14.5,False,None,None,No,No,No,Yes,False
1,1,91962954,Caucasian,Male,[80-90),NaN,2.0,1.0,7.0,3,...,B+,15.7,False,None,>7,No,No,No,No,True
2,3,157495374,African American,Female,[70-80),NaN,NaN,1.0,NaN,2,...,AB-,13.5,False,None,>8,No,No,No,Yes,False
3,4,82692360,Caucasian,Female,NaN,NaN,1.0,22.0,7.0,12,...,A+,13.0,False,None,None,No,No,No,No,False
4,5,218016576,Caucasian,Female,[70-80),NaN,2.0,1.0,1.0,4,...,A+,13.1,False,None,None,No,No,No,Yes,True


In [4]:

"""
data.age=data.age_as_int
data.discharge_disposition_code = data.discharge_disposition_simplified
data.admission_source_code = data.admission_source_simplified
data.medical_specialty = data.medical_specialty_simplified
data.diag_1 = data.diag_1_simplified
data.diag_2 = data.diag_2_simplified
data.diag_3 = data.diag_3_simplified
"""




'\ndata.age=data.age_as_int\ndata.discharge_disposition_code = data.discharge_disposition_simplified\ndata.admission_source_code = data.admission_source_simplified\ndata.medical_specialty = data.medical_specialty_simplified\ndata.diag_1 = data.diag_1_simplified\ndata.diag_2 = data.diag_2_simplified\ndata.diag_3 = data.diag_3_simplified\n'

In [5]:
data.columns[-7:]

Index(['max_glu_serum', 'A1Cresult', 'diuretics', 'insulin', 'change',
       'diabetesMed', 'readmitted'],
      dtype='object')

In [6]:
#dummy model: predicting everything as true
get_metrics(data.readmitted, [True]*len(data))

WRONGFUL DISCHARGE RATE: nan
F1_SCORE: 0.20393259701086341
RECALL: 1.0
PRECISION: 0.11354395535015278
ROC AUC: 0.5


In [7]:
#dummy model2 - predict same rate of readmission as in dataset
predict_same_rate = data.readmitted.sample(frac=1).reset_index(drop=True)
get_metrics(data.readmitted, predict_same_rate)

WRONGFUL DISCHARGE RATE: 0.11253240648414575
F1_SCORE: 0.12144128113879003
RECALL: 0.12144128113879003
PRECISION: 0.12144128113879003
ROC AUC: 0.5044544373273221


In [8]:
def diagnosis_decoder(code):
    if "V" in str(code): 
        return "External causes of injury and supplemental classification"
    elif "E" in str(code):
        return "External causes of injury and supplemental classification"
    else:
        try:
        
            code = int(code)
            if code<140: return "infectious and parasitic diseases"
            if code<240: return "neoplasms"
            if code<280: return "endocrine, nutritional and metabolic diseases, and immunity disorders"
            if code<290: return "diseases of the blood and blood-forming organs"
            if code<320: return "mental disorders"
            if code<390: return "diseases of the nervous system and sense organs"
            if code<460: return "diseases of the circulatory system"
            if code<520: return "diseases of the respiratory system"
            if code<580: return "diseases of the digestive system"
            if code<630: return "diseases of the genitourinary system"
            if code<680: return "complications of pregnancy, childbirth, and the puerperium"
            if code<710: return "diseases of the skin and subcutaneous tissue"
            if code<740: return "diseases of the musculoskeletal system and connective tissue"
            if code<760: return "congenital anomalies"
            if code<780: return "certain conditions originating in the perinatal period"
            if code<800: return "symptoms, signs, and ill-defined conditions"
            if code<1000: return "injury and poisoning"
        except:
            return(np.nan)
        




#### Feature engineering

In [9]:
#code age groups as integers
data["age_as_int"] = data.age.replace(['[50-60)', '[80-90)', '[60-70)', '[70-80)', '[40-50)', '[30-40)',
 '[90-100)', '[20-30)', '[10-20)', '[0-10)'], [50, 80, 60, 70, 40, 30, 90, 20, 10, 0])


In [10]:
#is patient insured
payer_codes = list(data.payer_code.dropna().unique())
payer_codes.remove("SP")

data["isInsured"] = data.payer_code.replace(list(payer_codes), True)
data["isInsured"] = data.isInsured.replace("SP", False)
data["isInsured"].unique()


array([nan, True, False], dtype=object)

In [11]:
def filter_common_categories(data, column_name, threshold):
    common_categories = list(data[column_name].value_counts()[data[column_name].value_counts()>threshold].index.values)
    common_categories.append(np.nan)
    data[column_name] = np.where(data[column_name].isin(common_categories), data[column_name], 'Other')
    data[column_name] = data[column_name].replace("nan", np.nan)
    return common_categories



In [12]:
#keep only common values for payer_code, set others as "Other"
column_name = "payer_code"
threshold=100
filter_common_categories(data, column_name, threshold)
#data[column_name].unique()

['MC',
 'HM',
 'SP',
 'BC',
 'MD',
 'CP',
 'UN',
 'CM',
 'OG',
 'PO',
 'DM',
 'CH',
 'WC',
 nan]

In [13]:
#keep only common values for admission_type_code, set others as "Other"
column_name = "admission_type_code"
threshold=100
filter_common_categories(data, column_name, threshold)
#data[column_name].unique()

[1.0, 3.0, 2.0, nan]

In [14]:
#keep only common values for discharge disposition, set others as "Other"
column_name = "discharge_disposition_code"
threshold=100
filter_common_categories(data, column_name, threshold)
#data[column_name].unique()

[1.0, 3.0, 6.0, 2.0, 22.0, 5.0, 4.0, 7.0, 23.0, 28.0, nan]

In [15]:
#keep only common values for admission_source_code, set others as "Other"
column_name = "admission_source_code"
threshold=100
filter_common_categories(data, column_name, threshold)
#data[column_name].unique()

[7.0, 1.0, 4.0, 6.0, 2.0, 5.0, 3.0, nan]

In [16]:
#keep only common values for medical_specialty set others as "Other"
column_name = "medical_specialty"
threshold=100
filter_common_categories(data, column_name, threshold)
#data[column_name].unique()

['InternalMedicine',
 'Emergency/Trauma',
 'Family/GeneralPractice',
 'Cardiology',
 'Surgery-General',
 'Nephrology',
 'Orthopedics',
 'Orthopedics-Reconstructive',
 'Radiologist',
 'Pulmonology',
 'Psychiatry',
 'Urology',
 'ObstetricsandGynecology',
 'Surgery-Cardiovascular/Thoracic',
 'Gastroenterology',
 'Surgery-Vascular',
 'Surgery-Neuro',
 'PhysicalMedicineandRehabilitation',
 'Oncology',
 'Pediatrics',
 'Neurology',
 'Hematology/Oncology',
 'Pediatrics-Endocrinology',
 'Otolaryngology',
 nan]

In [17]:
#simplify diagnosis codes
diag_columns = ['diag_1','diag_2','diag_3']
for col in diag_columns:
    data[f"{col}_simplified"] = data[col].str.replace(r"\.(.*)", "")  #remove any numbers that come after .
    #df_copy[col].str.replace('E','-')
    #df_copy[col] = df_copy[col].str.replace('V','-')
    data[f"{col}_simplified"] = data.apply(lambda row: diagnosis_decoder(row[f"{col}_simplified"]),axis=1)




In [18]:
data = data.drop(["admission_id", "patient_id", "age", "weight", "diag_1", "diag_2", "diag_3", "blood_type", "payer_code", "medical_specialty", "isInsured"], axis=1)

In [19]:
features_df = pd.DataFrame(list(zip((data.isnull().sum()/len(data)*100).values, data.nunique().values, data.dtypes.values)),  
    columns=["% of missing values", "Number of unique values", "Data type"], index=data.columns).drop("readmitted")

In [20]:
features_df

,% of missing values,Number of unique values,Data type
race,2.265323,5,object
gender,0.002525,2,object
admission_type_code,11.508448,4,object
discharge_disposition_code,5.381721,11,object
admission_source_code,6.907089,8,object
time_in_hospital,0.000000,14,int64
has_prosthesis,0.000000,2,bool
complete_vaccination_status,0.000000,3,object
num_lab_procedures,1.833472,115,float64
num_procedures,0.000000,7,int64


In [21]:
dfi.export(features_df, "features_used.png")


[0205/130346.961637:ERROR:gpu_init.cc(441)] Passthrough is not supported, GL is swiftshader
[0205/130347.533739:INFO:headless_shell.cc(648)] Written to file /tmp/tmpqu3ydg93/temp.png.


In [23]:
target = "readmitted"

all_features = list(features_df.index)
numerical_features = data.select_dtypes(include=['float64', 'int64']).columns
categorical_features = data.select_dtypes(include=['O', 'bool']).drop(["readmitted"], axis=1).columns

data[numerical_features] = data[numerical_features].astype(float)

In [24]:
df_train, df_test = train_test_split(data, test_size=0.3, random_state=42)



In [25]:
df_train.shape

(55435, 28)

In [32]:
classifiers = [
    LogisticRegression(random_state=42, n_jobs=-1, max_iter=1000),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42, n_jobs=-1),
    GradientBoostingClassifier(random_state=42),
    SVC(random_state=42)

    
]

parameters = [
              {"C": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100], "penalty": ['none', 'l1', 'l2', 'elasticnet']},

              {"max_depth": [1, 3, 5, 7, 9]},

              {'n_estimators': [100, 200, 500, 1000], 'max_depth' : [1, 3, 5, 7, 9]},

              {'n_estimators': [100, 200, 500, 1000], 'max_depth' : [1, 3, 5, 7, 9]},

               {'kernel': ["linear", "poly", "rbf", "sigmoid"], 'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]},


                

              
             ]

In [27]:



def define_pipeline(classifier, params):







    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value=np.nan)),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)])







    pipeline = make_pipeline(
        preprocessor,
        GridSearchCV(classifier,
                    param_grid=params,
                    cv=5, scoring="f1",
                    refit=True)

    )

    return pipeline










In [28]:
df_train = df_train.dropna()
X_train = df_train[all_features]
y_train = df_train[target]


# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X_train, y_train)


# define undersampling strategy
undersample = RandomUnderSampler(sampling_strategy='majority')
# fit and apply the transform
X_under, y_under = undersample.fit_resample(X_train, y_train)


In [29]:
print(X_train.shape, df_test.shape)

(40123, 27) (23759, 28)


In [30]:
##save sets
all_data = pd.read_csv("data/cleaned_data.csv")

#is patient insured
payer_codes = list(all_data.payer_code.dropna().unique())
payer_codes.remove("SP")

all_data["isInsured"] = all_data.payer_code.replace(list(payer_codes), True)
all_data["isInsured"] = all_data.isInsured.replace("SP", False)
all_data["isInsured"].unique()


df_train_to_save = df_train.merge(all_data.iloc[df_train.index][["age", "medical_specialty", "isInsured"]], left_index=True, right_index=True)
df_test_to_save = df_test.merge(all_data.iloc[df_test.index][["age", "medical_specialty", "isInsured"]], left_index=True, right_index=True)


df_train_to_save.to_csv("train_set.csv")
X_under.to_csv("train_set_under_X.csv")
y_under.to_csv("train_set_under_y.csv")
df_test_to_save.to_csv("test_set.csv")


In [33]:
best_params = []
for classifier, params in zip(classifiers, parameters):
    pipeline = define_pipeline(classifier, params)

    print(classifier)
    pipeline.fit(X_under, y_under)

    print(pipeline.named_steps['gridsearchcv'].best_params_)
     
    #make predictions
    X_test = df_test[all_features]

    y_test = df_test[target]

    y_pred = pipeline.predict(X_test)
    preds_proba = pipeline.predict_proba(X_test)[:, 1]

    get_metrics(y_test, y_pred)
    
    best_params.append(pipeline.named_steps['gridsearchcv'].best_params_)

    joblib.dump(pipeline, f'pipeline_{str(classifier)}.pickle')





LogisticRegression(max_iter=1000, n_jobs=-1, random_state=42)


/home/ana/.virtualenvs/capstone/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ana/.virtualenvs/capstone/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ana/.virtualenvs/capstone/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ana/.virtualenvs/capstone/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ana/.virtualenvs/capstone/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ana/.virtualenvs/ca

{'C': 0.01, 'penalty': 'l2'}
WRONGFUL DISCHARGE RATE: 0.07699947033898305
F1_SCORE: 0.2720872932066174
RECALL: 0.5706902916205242
PRECISION: 0.17862507221259388
ROC AUC: 0.6164852883280768
DecisionTreeClassifier(random_state=42)
{'max_depth': 5}
WRONGFUL DISCHARGE RATE: 0.07053197848176927
F1_SCORE: 0.25520092159653046
RECALL: 0.6950904392764858
PRECISION: 0.15629150066401062
ROC AUC: 0.6060962885218534
RandomForestClassifier(n_jobs=-1, random_state=42)
{'max_depth': 5, 'n_estimators': 500}
WRONGFUL DISCHARGE RATE: 0.07104282345782308
F1_SCORE: 0.26649650927107577
RECALL: 0.6552233296419343
PRECISION: 0.16726347531096872
ROC AUC: 0.6177066766974517
GradientBoostingClassifier(random_state=42)
{'max_depth': 3, 'n_estimators': 100}
WRONGFUL DISCHARGE RATE: 0.0733338178646704
F1_SCORE: 0.2675269494059328
RECALL: 0.6275378368401624
PRECISION: 0.17
ROC AUC: 0.616619274714618
SVC(random_state=42)
{'C': 1, 'kernel': 'rbf'}


AttributeError: predict_proba is not available when  probability=False

In [34]:
get_metrics(y_test, y_pred)
    
best_params.append(pipeline.named_steps['gridsearchcv'].best_params_)

joblib.dump(pipeline, f'pipeline_{str(classifier)}.pickle')

WRONGFUL DISCHARGE RATE: 0.0752883607130374
F1_SCORE: 0.2683548466661186
RECALL: 0.602436323366556
PRECISION: 0.17262534376983288
ROC AUC: 0.6154224372177197


['pipeline_SVC(random_state=42).pickle']

In [ ]:
best_params

[{'C': 0.01, 'penalty': 'l2'},
 {'max_depth': 5},
 {'max_depth': 5, 'n_estimators': 1000},
 {'max_depth': 3, 'n_estimators': 100},
 {'C': 1, 'kernel': 'rbf'}]

In [ ]:
hyperparameters_df = pd.DataFrame([best_params], columns=["LogisticRegression", "DecisionTreeClassifier", "RandomForestClassifier", "GradientBoostingClassifier", "SVC"])

In [ ]:
hyperparameters_df.to_csv("hyperparameters.csv")